In [1]:
import nltk
import pandas as pd 
import numpy as np 
import re
import random
from nltk.tokenize import sent_tokenize,word_tokenize,TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer #Haven't used
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
data  = pd.read_csv('https://raw.githubusercontent.com/MElHussieni/Twitter-US-Airline-Sentiment-Analysis/master/Tweets.csv')
data1 = pd.read_csv('airline_tweets_nov_08-nov_15.csv')
data2 = pd.read_csv('nlp_tweets_sample_11_16.csv')
data3 = pd.read_csv('airline_tweets_nov_17-nov_24.csv')
data4 = pd.read_csv('airline_tweets_nov_24-nov_27.csv')
data5 = pd.read_csv("airline_tweets_nov_25-Dec_02.csv")

In [3]:
data2 = data2.rename(columns={'entities_hashtag':'hashtags',   # renaming columns from api search to match kaggle data
                     'created_at': 'tweet_created',
                     'coordinates':'tweet_coord',
                     'full_text':'text',
                     'user_name':'name',
                     'id':'tweet_id'})  

#sampling 10% of all tweets collected per week to keep dataset size maneagable 

data1=data1.sample(frac=0.1, replace=True)                 
data2=data2.sample(frac=0.1, replace=True)
data3=data3.sample(frac=0.1, replace=True)
data4=data4.sample(frac=0.1, replace=True)
data5=data5.sample(frac=0.1, replace=True)

api_tweets = pd.concat([data1,data2,data3,data4,data5])        # merging API dataframes together together 

del api_tweets['user_screen_name']
del api_tweets['Unnamed: 0']

tweets=pd.concat([data,api_tweets])                     #merging kaggle and api tweets dataframes

In [4]:
words = [word_tokenize(tweet) for tweet in tweets[:]['text']]
sents = [sent_tokenize(tweet) for tweet in tweets[:]['text']]


In [5]:
#############################################

stop = stopwords.words('english')

tt = TweetTokenizer() #preform better on tweets

temp = tweets['text'].apply(tt.tokenize)

temp2 = temp.apply(lambda x: [word.lower() for word in x])

#need to be lowered to get rid of all the stopwords
temp2 = temp2.apply(lambda x: [word for word in x if word not in stop]) 
temp2 = temp2.apply(lambda x: [word for word in x if word not in stop]) 

wnl = WordNetLemmatizer()

tweets['full_text_processed'] = temp2.apply(lambda x: [wnl.lemmatize(word) for word in x]) 

tweets['full_text_processed'] = tweets['full_text_processed'].apply(lambda x: ' ' .join([word for word in x]))

###########################################################

In [6]:
processed_words=[word_tokenize(tweet) for tweet in tweets['full_text_processed']]

In [7]:
expletives_list = ['shit','fuck','damn','^hell(?!o)','crap','suck']
upper_count=[]
caps_ratio=[]
num_expletives=[] 
pun_ratio=[]
hashtags=[]

regex="^#"                                   # regular expression that matches hashtags

for word_list in words:                                 # iterates over every tweet
    hashtag='none'
    uppers=0                                             # counters to keep track of 
    expletives=0
    pun=0
    i=0
    for word in word_list:                              # iterates over every word in tweet
        
        if word.isupper():                               
            uppers+=1                                     # adds 1 to count if uppercase word
        
        if word.isalnum()==0:                             # adds files               
            pun+=1
            
        for expletive in expletives_list:                    # uses regex to find number of times people swear
            
            if bool(re.search(expletive,word.lower())):
                expletives+=1
            
                
        if bool(re.search(regex,word.lower())):           #word tokenize broke apart hashtags so we must look 
            try:
                hash_word=word_list[i+1].lower()                      # one word ahead to find hashtag
                if hashtag!=0:
                    hashtag='#'+hash_word+' '+hashtag 
                else:
                    hashtag='#'+hash_word
            except:
                continue
                
        i+=1
        
    ratio = uppers/len(word_list)                             # ratio of uppercase/(num_word)
    pun = pun/len(word_list)                                  # ratio of punctuation/(num_word)
    hashtags.append(hashtag)
    upper_count.append(uppers)
    num_expletives.append(expletives)
    caps_ratio.append(ratio)
    pun_ratio.append(pun)
    


In [8]:
tweets['sents'] = sents                    # sentences in each word
tweets['words'] = words                    # words in each tweets, NOT PROCESSED 
tweets['upper_count'] = upper_count        # number of words that are uppercase
tweets['caps_ratio'] = caps_ratio          # ratio of words that are uppercase
tweets['num_expletives'] = num_expletives  # number of expletives
tweets['pun_ratio'] = pun_ratio            # ratio of words that are punctuation 
tweets['clean_words'] = processed_words     # words in lowercase and after lemmanization 
tweets['hashtags'] = hashtags                # hashtags


In [9]:
#creating bag of words matrix for each tweet using processed text
matrix = CountVectorizer(max_features=2500)
X = matrix.fit_transform(tweets['full_text_processed']).toarray()

#making dataframe for bag of words and merging 
X = pd.DataFrame(X)
X.columns = matrix.get_feature_names() # 

X.reset_index(drop=True,inplace=True)
tweets.reset_index(drop=True,inplace=True)

tweets = pd.concat([tweets,X],axis=1)

#creating bag of words for hashtags
matrix = CountVectorizer(max_features=2500)
Y = matrix.fit_transform(tweets['hashtags']).toarray()
hash_list = matrix.get_feature_names()
hash_list = ['#'+hashtag for hashtag in hash_list]
Y = pd.DataFrame(Y)
Y.columns = hash_list
Y.reset_index(drop=True,inplace=True)

tweets = pd.concat([tweets,Y],axis=1)

In [10]:
labeled_data = tweets[0:14640]
api_data = tweets[14641:]

In [11]:
#labeled_data.to_csv("labeled_data.csv",sep=",",encoding='utf-8')
#api_data.to_csv("api_data.csv",sep=",",encoding='utf-8')
#tweets.to_csv("tweets_processed.csv",sep=",",encoding='utf-8')